# Machine learning using scikit-learn
> Best practices made simple with pipelines

In this notebook, we demonstrate the usage of pipelines to promote best practices in ML in python.  We'll make sure that all of our pre-processing steps are included within the cross validation training loop, measure performance using cross validation, and make sure to use literate programming practices to share our work.

# Technical objective
Our technical objective here is to predict the sex of a penguin given its other distinguishing features.

In [ ]:
#tables and visualizations
import pandas as pd
import numpy as np
import seaborn as sns

#machine learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn import config_context
from sklearn.metrics import classification_report

# Load the data
In this example, we'll use Allison Horst's penguins dataset.  Although this is normally used in R, we grab the csv from the penguins [GitHub repo](https://github.com/allisonhorst/palmerpenguins).  More information on the dataset is available on the introduction page [here.](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)

In [ ]:
#load the data and learn a bit
peng_data = pd.read_csv('https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/inst/extdata/penguins.csv')
display(peng_data.head())
peng_data.info()

# Data cleaning and EDA
We can now explore our data.  We leave this exercise to the reader.  For now, we can observe that there are a few NA values which will likely need imputation.  We'll wait for this step so that we can put it within our training loop.  For now, we'll just drop all of the sex NAs out of the dataframe.

In [ ]:
peng_data = peng_data.dropna(subset=['sex'])

In [ ]:
peng_data.shape

This set of dimensions is expected.

# Split the data
Here, we employ the initial split to separate the training from the golden holdout test set.

In [ ]:
class_column = 'sex'
random_seed = 2435

X_train, X_test, y_train, y_test = train_test_split(peng_data.drop(columns=class_column), peng_data[class_column],
                                                   test_size=0.25, random_state=random_seed, stratify=peng_data[class_column])

Quick sanity check to make sure that everything seems OK:

In [ ]:
# X Train
print('On X train: ')
print('X train dimensions: ', X_train.shape)
display(X_train.head())

# X test
print('\nOn X test: ')
print('X test dimensions: ', X_test.shape)
display(X_test.head())

In [ ]:
# X Train
print('On y train: ')
print('y train dimensions: ', y_train.shape)
display(y_train.head())

# X test
print('\nOn y test: ')
print('y test dimensions: ', y_test.shape)
display(y_test.head())

I love a good sanity check.  Here, we can see that the ratios of the split are about correct.  We can also observe that the indices match (for example, between X and y train).  These are all good indicators.

# Establish training pipeline
It looks like we first might need to do some imputation on some values within our rows.  Then, it looks like we have some categorical columns so those will need to be encoded.  However, this isn't all the columns since our column types are heterogeneous.  Let's see what transforming specific columns looks like here.

Note that in the end, this will be methodology-specific.  Although we use logistic regression below, we also perform imputation to demonstrate the method.

In [ ]:
#individual pipelines for differing datatypes
cat_pipeline = Pipeline(steps=[('cat_impute', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                               ('onehot_cat', OneHotEncoder(drop='if_binary'))])
num_pipeline = Pipeline(steps=[('impute_num', SimpleImputer(missing_values=np.nan, strategy='mean')),
                               ('scale_num', StandardScaler())])     

In [ ]:
#establish preprocessing pipeline by columns
preproc = ColumnTransformer([('cat_pipe', cat_pipeline, make_column_selector(dtype_include=object)),
                             ('num_pipe', num_pipeline, make_column_selector(dtype_include=np.number))],
                             remainder='passthrough')

In [ ]:
#generate the whole modeling pipeline with preprocessing
pipe = Pipeline(steps=[('preproc', preproc),
                       ('mdl', LogisticRegression(penalty='elasticnet', solver='saga', tol=0.01))])

#visualization for steps
with config_context(display='diagram'):
    display(pipe)

# Cross-validation with hyperparameter tuning

In [ ]:
tuning_grid = {'mdl__l1_ratio' : np.linspace(0,1,5),
               'mdl__C': np.logspace(-1, 6, 3) }
grid_search = GridSearchCV(pipe, param_grid = tuning_grid, cv = 5, return_train_score=True)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_score_)
grid_search.best_params_

In [ ]:
pd.DataFrame(grid_search.cv_results_)

# Final fit
The final fit here is already present in the generated model due to the way we set our parameters in the grid search.  If we want to look at the performance, we can do so.  Here is a non-helpful description of the best model:

In [ ]:
grid_search.best_estimator_

# Variable importance
Now we assess the importance in the selected model to reveal any potential insights.

In [ ]:
grid_search.classes_

In [ ]:
vip = grid_search.best_estimator_['mdl'].coef_[0]
vip

In [ ]:
#get names in correct preproc order
cat_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[0][1].named_steps['onehot_cat'].get_feature_names()
num_names = grid_search.best_estimator_.named_steps['preproc'].transformers_[1][2]

#create df with vip info
coef_info = pd.DataFrame({'feat_names':np.hstack([cat_names, num_names]), 'vip': vip})

#get sign and magnitude information
coef_info = coef_info.assign(coef_mag = abs(coef_info['vip']),
                             coef_sign = np.sign(coef_info['vip']))

#sort and plot
coef_info = coef_info.set_index('feat_names').sort_values(by='coef_mag', ascending=False)
sns.barplot(y=coef_info.index, x='coef_mag', hue='coef_sign', data=coef_info, orient='h', dodge=False);

# Performance metrics on test data

In [ ]:
print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

Here, we can see the performance of the model, which is pretty nice!  We can also look into different scores specifically for more insight into the performance.